### Тестовое задание от крупной компании
Ниже выполнено тестовое задание от некоторой крупной компании. Было необходимо решить 3 задачи из 10 предложенных, в итоге, решено 7 задач. Вначале сгенерирован датафрейм со случайными данными. В том числе, сгенерирована воронка по событиям "посещение сайта" и "открытие счета". Т.е. некоторые пользователи зашли на сайт, но не открыли счет, а некоторые - зашли на сайт и открыли счет. Далее идут решения задач.

### Подготовка таблицы

In [1]:
# Импорт библиотек:

import pandas as pd
!pip install pandasql
from pandasql import sqldf
import numpy as np
from numpy.random import choice
import random

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/tljh/user/bin/python -m pip install --upgrade pip' command.


<span style="color:blue">Библиотеки импортированы.</span>

In [2]:
# Необходимо сгенерировать таблицу events со следующими столбцами:

# event_datetime datetime,  время события, только 2020 год
# event_id int,             идентификатор события, первичный ключ

# user_id int,              идентификатор посетителя (один пользователь может несколько раз заходить на сайт, в том числе 
#                           и в течение дня, в том числе с разных городов). Посещения могут быть уже после открытия счета.

# event_name varchar(100),  название события (Посещение сайта или Открытие счета) Для упрощения - счет может открыться 
#                           только после посещения сайта, и каждый посетитель мождет открыть только один счет

# city  varchar(100) ,      город
# cost  numeric(10,2)       расчетная стоимость привлечения посетителя на сайт


<span style="color:blue">Далее будет сгенерирован необходимый датафрейм.</span>

In [3]:
# Зададим данные для столбцов event_id и user_id:

# создадим последовательности чисел
all_events_ids = np.arange(1, 1001)
all_users_ids = np.arange(1, 301)

# создадим случайный выбор 1000 чисел из вышеуказанных последовательностей
event_ids    = np.random.choice(all_events_ids, 1000)
user_ids = np.random.choice(all_users_ids, 1000)

<span style="color:blue">Данные для столбцов event_id и user_id заданы.</span>

In [4]:
# Зададим даты для столбца event_datetime:

# укажем стартовую дату 01.01.2020
start_date = pd.to_datetime('2020-01-01')

# зададим 1000 дат, начиная со start_date, с интервалом 5 часов
event_datetime = pd.date_range(start_date, periods=1000, freq='5H')

<span style="color:blue">Даты для столбца event_datetime заданы.</span>

In [5]:
# Создадим датафрейм:

user_actions = pd.DataFrame({'event_datetime': event_datetime,
                             'event_id': event_ids,
                             'user_id': user_ids})
user_actions

,event_datetime,event_id,user_id
0,2020-01-01 00:00:00,909,201
1,2020-01-01 05:00:00,791,120
2,2020-01-01 10:00:00,493,45
3,2020-01-01 15:00:00,538,252
4,2020-01-01 20:00:00,568,230
...,...,...,...
995,2020-07-26 07:00:00,813,238
996,2020-07-26 12:00:00,842,88
997,2020-07-26 17:00:00,157,272
998,2020-07-26 22:00:00,115,95


<span style="color:blue">Часть датафрейма с датами, event_id и user_id создана.</span>

In [6]:
# Создадим столбец "действие" с посещением сайта, т.к. все пользователи сначала заходили на сайт:

user_actions['event_name'] = 'Посещение сайта'
user_actions.head()

,event_datetime,event_id,user_id,event_name
0,2020-01-01 00:00:00,909,201,Посещение сайта
1,2020-01-01 05:00:00,791,120,Посещение сайта
2,2020-01-01 10:00:00,493,45,Посещение сайта
3,2020-01-01 15:00:00,538,252,Посещение сайта
4,2020-01-01 20:00:00,568,230,Посещение сайта


<span style="color:blue">Столбец event_name заполнен только событием "Посещение сайта".</span>

In [7]:
# Теперь сгерерируем событие "Открытие счета" для некоторых пользователей,
# для чего сначала напишем функцию для однократной генерации строки с событием 'Открытие счета': 

def generate_funel_actions(event_datetime, event_id, user_id):
    to_account = 0.3 # вероятность успеха события 'Открытие счета'
    
    
    df = pd.DataFrame()
    
    # в случае успеха функция создаст событие 'Открытие счета'
    if np.random.binomial(1, to_account, 1)[0]:
        # одна строка с событием 'Открытие счета'
        df = pd.DataFrame({'event_datetime': event_datetime + pd.Timedelta(5, unit='s'),
                           'event_id': event_id,
                           'user_id': user_id,                       
                           'event_name': 'Открытие счета'}, index=[0])
        
            
    return df

<span style="color:blue">Функция создана. Задача функции - сгенерировать событие "Открытие счета" для каждой строки датафрейма с вероятностью 0.3.</span>

In [8]:
# Применим созданную функцию к user_actions:

# создадим пустой датафрейм, чтобы заполнить его событиями "Открытие счета"
open_account = pd.DataFrame()

# идем по индексам и строкам в user_actions
for index, row in user_actions.iterrows():
    # передаем функции значения каждой строки из user_actions    
    user_df = generate_funel_actions(row['event_datetime'], row['event_id'], row['user_id'])
    # датафрейм с событиями 'Открытие счета'
    open_account = open_account.append(user_df)
    
#  создадим общий датафрейм со всеми событиями, строки суммируются
user_actions = user_actions.append(open_account)
user_actions = user_actions.sort_values('event_datetime')
display(user_actions.head())
print(user_actions['event_name'].value_counts())

,event_datetime,event_id,user_id,event_name
0,2020-01-01 00:00:00,909,201,Посещение сайта
1,2020-01-01 05:00:00,791,120,Посещение сайта
2,2020-01-01 10:00:00,493,45,Посещение сайта
3,2020-01-01 15:00:00,538,252,Посещение сайта
4,2020-01-01 20:00:00,568,230,Посещение сайта


Посещение сайта    1000
Открытие счета      279
Name: event_name, dtype: int64


<span style="color:blue">Столбец event_name сгенерирован. Выше указано количество созданных строк с событием "Открытие счета"</span>

In [9]:
# Создадим столбец с названиями городов в виде одной буквы от "а" до "м":

user_actions['city'] = [''.join(random.choice('абвгдежзиклм') for i in range(1)) for _ in range(user_actions.shape[0])]
user_actions.head()

,event_datetime,event_id,user_id,event_name,city
0,2020-01-01 00:00:00,909,201,Посещение сайта,м
1,2020-01-01 05:00:00,791,120,Посещение сайта,л
2,2020-01-01 10:00:00,493,45,Посещение сайта,а
3,2020-01-01 15:00:00,538,252,Посещение сайта,г
4,2020-01-01 20:00:00,568,230,Посещение сайта,б


<span style="color:blue">Столбец "city" создан.</span>

In [10]:
# Создадим столбец со стоимостью привлечения в виде числа от 1 до 9:

user_actions['cost'] = [''.join(random.choice('123456789') for i in range(1)) for _ in range(user_actions.shape[0])]
user_actions.head()

,event_datetime,event_id,user_id,event_name,city,cost
0,2020-01-01 00:00:00,909,201,Посещение сайта,м,2
1,2020-01-01 05:00:00,791,120,Посещение сайта,л,9
2,2020-01-01 10:00:00,493,45,Посещение сайта,а,2
3,2020-01-01 15:00:00,538,252,Посещение сайта,г,9
4,2020-01-01 20:00:00,568,230,Посещение сайта,б,6


<span style="color:blue">Столбец "cost" создан. Датафрейм готов, можно приступать к решению задач.</span>

### Решения задач
Ниже приведены решения 7 задач из 10 предложенных в виде запросов SQL

In [11]:
# Задача 1. Вывести количество уникальных посетителей, которые заходили на сайт из разных городов:

q1 = """   SELECT COUNT(DISTINCT user_id) users_cnt_from_dif_cities
           FROM
           (SELECT user_id,
                   COUNT(DISTINCT city) 
           FROM user_actions
           WHERE event_name = 'Посещение сайта'
           GROUP BY user_id
           HAVING COUNT(DISTINCT city)>1);"""


In [12]:
sqldf(q1)

,users_cnt_from_dif_cities
0,252


In [13]:
# Задача 2. Найти город с максимальным числом уникальных посетителей. Вывести в разрезе каждого месяца:

q2 = """    SELECT month, city, MAX(users_cnt) users_cnt_max
            FROM
            (SELECT strftime('%m', event_datetime) AS month,
                   city,
                   COUNT(DISTINCT user_id) AS users_cnt 
            FROM user_actions
            WHERE event_name = 'Посещение сайта'
            GROUP BY month, city)
            GROUP BY month;"""

In [14]:
sqldf(q2)

,month,city,users_cnt_max
0,01,е,19
1,02,ж,21
2,03,г,16
3,04,б,15
4,05,м,20
5,06,г,15
6,07,и,14


In [15]:
# Задача 3. Найти среднее количество посещений сайта, потребовавшееся посетителям, 
# чтобы принять решение об открытии счета (результат = одно число, только по посетителям, открывшим счет):

q3 = """    SELECT ROUND(AVG(visits_cnt), 2) visits_cnt_mean FROM
            (SELECT user_id,
                    COUNT(event_id) visits_cnt
            FROM user_actions
            WHERE event_name = 'Посещение сайта' 
            AND user_id IN
            (SELECT user_id
                    FROM user_actions
            WHERE event_name = 'Открытие счета'
            GROUP BY user_id) 
            AND event_datetime <
            (SELECT event_datetime
                   FROM user_actions
            WHERE event_name = 'Открытие счета'
            GROUP BY user_id)
            GROUP BY user_id);"""

In [16]:
sqldf(q3)

,visits_cnt_mean
0,3.46


In [17]:
# Задача 5. Вывести идентификаторы посетителей, которые открыли счет в день первого посещения сайта:

q5 = """   SELECT user_id
            FROM user_actions
            WHERE 
            (SELECT STRFTIME('%Y-%m-%d', event_datetime)      
            FROM user_actions
            WHERE event_name = 'Открытие счета'
            GROUP BY user_id) = 
            (SELECT MIN(STRFTIME('%Y-%m-%d', event_datetime))
            FROM user_actions
            GROUP BY user_id)
            
            AND user_id IN
            (SELECT user_id     
            FROM user_actions
            WHERE event_name = 'Открытие счета'
            GROUP BY user_id)
            GROUP BY user_id;"""

In [18]:
sqldf(q5)

,user_id


In [19]:
# Задача 6. Найти общую стоимость привлечения и количество посетителей, которые в течение года так и не  открыли счет:

q6 = """    SELECT COUNT(user_id) users_cnt,
                   SUM(cost_per_user) total_cost
            FROM
            (SELECT user_id,
                   SUM(cost) cost_per_user
            FROM user_actions
            WHERE user_id NOT IN
            (SELECT user_id
                    FROM user_actions
            WHERE event_name = 'Открытие счета'
            GROUP BY user_id)
            GROUP BY user_id);"""

In [20]:
sqldf(q6)

,users_cnt,total_cost
0,101,1385


In [21]:
# Задача 8. Найти общую стоимость привлечения и количество посетителей, которые в течение года открыли счет:

q8 = """    SELECT COUNT(user_id) users_cnt,
                   SUM(cost_per_user) total_cost
            FROM
            (SELECT user_id,
                   SUM(cost) cost_per_user
            FROM user_actions
            WHERE user_id IN
            (SELECT user_id
                    FROM user_actions
            WHERE event_name = 'Открытие счета'
            GROUP BY user_id)
            GROUP BY user_id);"""

In [22]:
sqldf(q8)

,users_cnt,total_cost
0,186,5096


In [23]:
# Задача 9. Вывести уникальные даты посещения сайта посетителем, стоимость привлечения которого на сайт 
# (Событие "Посещение сайта", за весь год) оказалась самой высокой. Независимо от того, открыл или не открыл счет:

q9 = """    SELECT DISTINCT STRFTIME('%Y-%m-%d', event_datetime) unique_dates
            FROM user_actions
            WHERE user_id =
            (SELECT user_id                  
            FROM user_actions
            WHERE event_name = 'Посещение сайта'
            GROUP BY user_id
            ORDER BY SUM(cost) DESC LIMIT 1);"""

In [24]:
sqldf(q9)

,unique_dates
0,2020-01-08
1,2020-01-31
2,2020-02-13
3,2020-03-11
4,2020-03-19
5,2020-03-29
6,2020-04-01
7,2020-04-23
8,2020-05-07
9,2020-06-20


<span style="color:blue">Решены 7 задач.</span>